In [3]:
import numpy as np
import pandas as pd
import math

In [4]:
df = pd.read_csv('sent_receive.csv')
sent_id, receive_id = df['sent_id'].values, df['receive_id'].values

In [5]:
trans = {}
num_set = list((set(sent_id) | set(receive_id)))
# trans 从原序号映射到新结点序号0-...
for i in range(len(num_set)):
    trans.update({num_set[i] : i})

In [20]:
M = np.zeros(shape=(len(num_set), len(num_set)))
for i in range(len(sent_id)):
    M[trans[sent_id[i]]][trans[receive_id[i]]] = 1

In [10]:
R = np.full(len(num_set), 1 / len(num_set))

In [22]:
for i in range(len(M)):
    if np.sum(M[i]):
        M[i] = np.divide(M[i], np.sum(M[i]))

In [24]:
M = M.T

In [28]:
def update_mat(mat):
    col_sum = np.sum(mat, axis=0)
    n, beta = len(mat), 0.85
    ind = [i for i in range(len(col_sum)) if col_sum[i] == 0]
    for i in range(len(mat)):
        for j in ind:
            mat[i][j] = 1 / n
    mat = np.multiply(mat, beta)
    mat += np.multiply(np.full(mat.shape, 1 / n), 1 - beta)
    return mat

In [29]:
M = update_mat(M)
R = R.reshape((len(R), 1))

In [39]:
# 计算两个n*1列向量的距离
def cal_distance(r1, r2):
    return np.sqrt(np.sum(np.square(r1 - r2)))

In [49]:
new_R = np.dot(M, R)
epsilon = math.pow(10, -8)
while cal_distance(new_R, R) >= epsilon:
    R = new_R
    new_R = np.dot(M, R)

In [57]:
with open('ans', 'w') as w:
    for i in range(len(new_R)):
        w.write(str(num_set[i]) + '\t' + str(new_R[i][0]) + '\n')
w.close()